In [1]:
from qem.fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'
image = np.loadtxt(file)
dx= 0.1645429228960236	
model=ImageModelFitting(image, dx=dx,units='A')

# get the unique coordinates by row
# input_coordinates = np.unique(input_coordinates, axis=0)

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
model.find_peaks()

model.select_region(invert_selection=False)

Updated peak locations.


INFO:root:Selected 6734 atoms out of 7300 atoms.


In [3]:
from qem.crystal_analyzer import CrystalAnalyzer

crystal_analyzer = CrystalAnalyzer(image = model.image, dx = model.dx, peak_positions = model.coordinates, atom_types = model.atom_types, elements=['Sr','Ti'],units=model.units)
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
crystal_analyzer.read_cif(cif_file)
crystal_analyzer.choose_lattice_vectors(tolerance=20)
supercell_in_image, supercell_atom_types = crystal_analyzer.generate_supercell_lattice(a_limit=5, b_limit=5)

Origin selected: [1181. 1177.]
Vector a selected: [24. -2.]
Vector b selected: [ -2. -28.]
In pixel: Origin: [1181. 1177.], Vector a: [24. -2.], Vector b: [ -2. -28.]
In space: Origin: [194.32519194 193.66702025] A, Vector a: [ 3.94903015 -0.32908585] A, Vector b: [-0.32908585 -4.60720184] A
Origin selected: [1024 1024]
Vector a selected: [84  0]
Vector b selected: [ 0 84]
In pixel: Origin: [1024 1024], Vector a: [84  0], Vector b: [ 0 84]
In space: Origin: [3.03872079 3.03872079] 1/A, Vector a: [0.24927006 0.        ] 1/A, Vector b: [0.         0.24927006] 1/A
FFT real a: [24.38095238  0.        ] pixel, Real b: [ 0.         24.38095238] pixel


In [8]:
plt.imshow(model.image)
plt.scatter(supercell_in_image[:,0], supercell_in_image[:,1], c='r', s=100)

In [4]:
crystal_analyzer.plot()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 6823 but corresponding boolean dimension is 121

In [3]:
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'],min_distance=20,a_limit=50, b_limit=50 )

Origin selected: [1206. 1030.]
Vector a selected: [24.  0.]
Vector b selected: [ -1. -26.]
In pixel: Origin: [1206. 1030.], Vector a: [24.  0.], Vector b: [ -1. -26.]
In space: Origin: [198.43876501 169.47921058] A, Vector a: [3.94903015 0.        ] A, Vector b: [-0.16454292 -4.278116  ] A
Origin selected: [1024 1024]
Vector a selected: [84  0]
Vector b selected: [ 0 84]
In pixel: Origin: [1024 1024], Vector a: [84  0], Vector b: [ 0 84]
In space: Origin: [3.03872079 3.03872079] 1/A, Vector a: [0.24927006 0.        ] 1/A, Vector b: [0.         0.24927006] 1/A
FFT real a: [24.38095238  0.        ] pixel, Real b: [ 0.         24.38095238] pixel


In [4]:
model.select_region()

INFO:root:Selected 13259 atoms out of 14130 atoms.


In [5]:
model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=50, tol=1e-3, plot=False, step_size=0.01,verbose=False,num_epoch=1)

Fitting random batch:   0%|          | 0/14 [00:00<?, ?it/s]INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
Fitting random batch: 100%|██████████| 14/14 [01:02<00:00,  4.45s/it]
INFO:root:Convergence rate for pos_x = 0.5816163562594738
INFO:root:Convergence rate for pos_y = 0.6033935546875
INFO:root:Convergence rate for height = 0.0014269453991787612
INFO:root:Convergence not reached


{'pos_x': array([2.18711317e-01, 9.45631504e+00, 9.48135853e+00, ...,
        2.04700000e+03, 2.04700000e+03, 2.04700000e+03]),
 'pos_y': array([133.5970459 , 164.14364624, 237.33305359, ..., 505.67932129,
        532.09179688, 653.67700195]),
 'height': array([ 72.93719264, 563.60702213, 499.36819532, ..., 325.64941576,
        358.34907731, 348.68502882]),
 'sigma': array([4.22239968, 4.23322198, 4.23322198, ..., 4.22239968, 4.22239968,
        4.22239968]),
 'background': 11545.89856141919}

In [6]:
model.plot_fitting()

In [1]:
from qem.crystal_analyzer import CrystalAnalyzer
from qem.fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'
image = np.loadtxt(file)
dx= 0.1645429228960236	
model=ImageModelFitting(image, dx=dx,units='A')
model.find_peaks()
crystal_analyzer = CrystalAnalyzer(image = model.image, dx = model.dx, peak_positions = model.coordinates, atom_types = model.atom_types, elements=['Sr','Ti'])

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Updated peak locations.


INFO:root:No atoms selected.


In [6]:

cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
crystal_analyzer.read_cif(cif_file)

Atoms(symbols='SrTi', pbc=True, cell=[3.905, 3.905, 3.905], spacegroup_kinds=...)

In [4]:
crystal_analyzer.choose_lattice_vectors()

Selection incomplete.
Selection incomplete.


TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'

In [5]:
crystal_analyzer._origin_adaptive = None
shift_center = crystal_analyzer.shift_origin_adaptive(50, 50)
plt.imshow(model.image, cmap='gray')
peaks = []
for key, value in shift_center.items():
    peaks.append(value)

plt.scatter(np.array(peaks)[:,0], np.array(peaks)[:,1], c='r', s=10)
plt.show()

In [6]:
peak_positions, atom_types = crystal_analyzer.generate_supercell_lattice(a_limit=50, b_limit=50)

In [14]:
peak_positions, atom_types = crystal_analyzer.supercell_project_2d()

In [7]:
plt.scatter(peak_positions[:,0], peak_positions[:,1], c='r', s=10)

In [10]:
crystal_analyzer.unitcell_mapping()

TypeError: list indices must be integers or slices, not numpy.float64

In [23]:
im_refined = model.predict_local(params=model.params)
plt.imshow(im_refined)

In [25]:
plt.imshow(model.image-im_refined)

In [18]:
model.atom_types

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,